<a href="https://colab.research.google.com/github/suribe1010/Kaggle/blob/master/ConvolutionalNN__RESNET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

THIS IS AN EXAMPLE OF A CONVOLUTIONAL NEURAL NETWORK WITH RESNET ARCHITECTURE


In [0]:
# IMPORT PACKAGES

import keras
from keras.datasets import cifar10
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,AveragePooling2D,Dropout,BatchNormalization,Activation
from keras.models import Model,Input
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras.callbacks import ModelCheckpoint
from math import ceil
import os
from keras.preprocessing.image import ImageDataGenerator

In [0]:
# LOAD THE CIFAR10 DATASET

(train_x, train_y) , (test_x, test_y) = cifar10.load_data()

In [3]:
train_x.shape

(50000, 32, 32, 3)

In [4]:
train_x[0]

array([[[ 59,  62,  63],
        [ 43,  46,  45],
        [ 50,  48,  43],
        ...,
        [158, 132, 108],
        [152, 125, 102],
        [148, 124, 103]],

       [[ 16,  20,  20],
        [  0,   0,   0],
        [ 18,   8,   0],
        ...,
        [123,  88,  55],
        [119,  83,  50],
        [122,  87,  57]],

       [[ 25,  24,  21],
        [ 16,   7,   0],
        [ 49,  27,   8],
        ...,
        [118,  84,  50],
        [120,  84,  50],
        [109,  73,  42]],

       ...,

       [[208, 170,  96],
        [201, 153,  34],
        [198, 161,  26],
        ...,
        [160, 133,  70],
        [ 56,  31,   7],
        [ 53,  34,  20]],

       [[180, 139,  96],
        [173, 123,  42],
        [186, 144,  30],
        ...,
        [184, 148,  94],
        [ 97,  62,  34],
        [ 83,  53,  34]],

       [[177, 144, 116],
        [168, 129,  94],
        [179, 142,  87],
        ...,
        [216, 184, 140],
        [151, 118,  84],
        [123,  92,  72]]

In [5]:
print('Shape of each element (32 elements) of 1st example: ', train_x[0][0].shape)
train_x[0][0]
train_x[0][31]

Shape of each element (32 elements) of 1st example:  (32, 3)


array([[177, 144, 116],
       [168, 129,  94],
       [179, 142,  87],
       [188, 149,  67],
       [202, 168,  68],
       [218, 189,  76],
       [218, 191,  72],
       [207, 181,  70],
       [191, 163,  79],
       [175, 143,  82],
       [166, 132,  86],
       [163, 128,  92],
       [163, 127,  94],
       [161, 123,  92],
       [153, 114,  84],
       [159, 120,  90],
       [162, 124,  93],
       [149, 116,  91],
       [140, 104,  83],
       [148, 103,  77],
       [161, 105,  69],
       [144,  95,  55],
       [112,  90,  59],
       [119,  91,  58],
       [130,  96,  65],
       [120,  87,  59],
       [ 92,  67,  46],
       [103,  78,  57],
       [170, 140, 104],
       [216, 184, 140],
       [151, 118,  84],
       [123,  92,  72]], dtype=uint8)

There are 32 matrixes of size (32,3) as the one before in 1 example.
There are 50000 examples.

In [32]:
# NORMALIZATION OF THE DATA

train_x = train_x.astype('float32') / 255
test_x = test_x.astype('float32') / 255

train_x = train_x - train_x.mean()
test_x = test_x - test_x.mean()

train_x = train_x / train_x.std(axis=0)
test_x = test_x / test_x.std(axis=0)

train_x[0][0]

array([[-0.84057784, -0.8052632 , -0.71733737],
       [-1.0727214 , -1.0390277 , -0.95115703],
       [-0.97879755, -1.0144184 , -0.97827077],
       [-0.7318775 , -0.9332892 , -0.99340624],
       [-0.31664565, -0.67041355, -0.8701512 ],
       [-0.02387652, -0.41884312, -0.73228234],
       [ 0.25636435, -0.19378802, -0.5810944 ],
       [ 0.34079367, -0.15148064, -0.51818764],
       [ 0.39808214, -0.05263771, -0.40490326],
       [ 0.39828354, -0.01005867, -0.3538893 ],
       [ 0.14522853, -0.2522744 , -0.5592571 ],
       [ 0.06061345, -0.309664  , -0.5724796 ],
       [ 0.30122873, -0.08149043, -0.3805845 ],
       [ 0.3300301 , -0.12446555, -0.4451826 ],
       [ 0.23101898, -0.22474778, -0.5356897 ],
       [ 0.11756594, -0.33905846, -0.63824314],
       [ 0.23092982, -0.21028787, -0.53551054],
       [ 0.18834722, -0.2102308 , -0.57388633],
       [ 0.04662099, -0.33857846, -0.7271696 ],
       [ 0.258721  , -0.10995047, -0.5466715 ],
       [ 0.2578694 , -0.12373962, -0.583

# DATA AUGMENTATION ????
Antes de continuar, merece la pena explicar qué es data augmentation. La idea es que, cuando se dispone de un número de imágenes relativamente pequeño, podemos aumentar el número modificando las imágenes originales (haciendo zoom, escalado, flip horizontal, etc)


In [0]:
# Rotándola 10 grados y haciendo un flip horizontal.
datagen = ImageDataGenerator(rotation_range=10,
                             width_shift_range=5. / 32,
                             height_shift_range=5. / 32,
                             horizontal_flip=True)


In [0]:
datagen.fit(train_x)

In [35]:
print('Shape of train labeled data: ', train_y.shape)
train_y

Shape of train labeled data:  (50000, 1)


array([[6],
       [9],
       [9],
       ...,
       [9],
       [1],
       [1]], dtype=uint8)

In [36]:
# ENCODE THE LABELS TO VECTORS
train_y = keras.utils.to_categorical(train_y,10)
test_y = keras.utils.to_categorical(test_y,10)
print('Shape of train labeled data: ', train_y.shape)
train_y[0]

Shape of train labeled data:  (50000, 10)


array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.], dtype=float32)

In [0]:
# Define the model

def Unit(x,filters,pool=False):
    res = x
    if pool:
        x = MaxPooling2D(pool_size=(2, 2))(x)
        res = Conv2D(filters=filters,kernel_size=[1,1],strides=(2,2),padding="same")(res)
    
    out = BatchNormalization()(x)
    out = Activation("relu")(out)
    out = Conv2D(filters=filters, kernel_size=[3, 3], strides=[1, 1], padding="same")(out)

    out = BatchNormalization()(out)
    out = Activation("relu")(out)
    out = Conv2D(filters=filters, kernel_size=[3, 3], strides=[1, 1], padding="same")(out)

    out = keras.layers.add([res,out])

    return out


def MiniModel(input_shape):
    images = Input(input_shape)
    net = Conv2D(filters=32, kernel_size=[3, 3], strides=[1, 1], padding="same")(images)
    net = Unit(net,32)
    net = Unit(net,32)
    net = Unit(net,32)

    net = Unit(net,64,pool=True)
    net = Unit(net,64)
    net = Unit(net,64)

    net = Unit(net,128,pool=True)
    net = Unit(net,128)
    net = Unit(net,128)

    net = Unit(net, 256,pool=True)
    net = Unit(net, 256)
    net = Unit(net, 256)

    net = BatchNormalization()(net)
    net = Activation("relu")(net)
    net = Dropout(0.25)(net)

    net = AveragePooling2D(pool_size=(4,4))(net)
    net = Flatten()(net)
    net = Dense(units=10,activation="softmax")(net)

    model = Model(inputs=images,outputs=net)

    return model



In [37]:
input_shape = (32,32,3)
model = MiniModel(input_shape)
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_65 (Conv2D)              (None, 32, 32, 32)   896         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_57 (BatchNo (None, 32, 32, 32)   128         conv2d_65[0][0]                  
__________________________________________________________________________________________________
activation_57 (Activation)      (None, 32, 32, 32)   0           batch_normalization_57[0][0]     
____________________________________________________________________________________________

In [0]:
# Training components

model.compile(optimizer=Adam(0.001),loss="categorical_crossentropy",metrics=["accuracy"])


In [0]:

epochs = 50
steps_per_epoch = ceil(50000/128)

In [40]:
# Fit the model on the batches generated by datagen.flow()

model.fit_generator(datagen.flow(train_x, train_y, batch_size=128),
                    validation_data=[test_x,test_y],
                    epochs=epochs,steps_per_epoch=steps_per_epoch, verbose=1, workers=4)

Epoch 1/50
391/391 [==============================] - 2466s 6s/step - loss: 1.4440 - acc: 0.4654 - val_loss: 1.6360 - val_acc: 0.4669
Epoch 2/50
391/391 [==============================] - 2457s 6s/step - loss: 0.9652 - acc: 0.6562 - val_loss: 1.6405 - val_acc: 0.5252
Epoch 3/50
391/391 [==============================] - 2462s 6s/step - loss: 0.7689 - acc: 0.7293 - val_loss: 0.8673 - val_acc: 0.7198
Epoch 4/50
391/391 [==============================] - 2455s 6s/step - loss: 0.6577 - acc: 0.7706 - val_loss: 0.9322 - val_acc: 0.7087
Epoch 5/50
391/391 [==============================] - 2461s 6s/step - loss: 0.5797 - acc: 0.7971 - val_loss: 0.9125 - val_acc: 0.7094
Epoch 6/50
391/391 [==============================] - 2461s 6s/step - loss: 0.5266 - acc: 0.8163 - val_loss: 0.7305 - val_acc: 0.7580
Epoch 7/50
391/391 [==============================] - 2463s 6s/step - loss: 0.4895 - acc: 0.8290 - val_loss: 0.8043 - val_acc: 0.7509
Epoch 8/50
391/391 [==============================] - 2460s 6s

KeyboardInterrupt: ignored

In [41]:
#Evaluate the accuracy of the test dataset
accuracy = model.evaluate(x=test_x,y=test_y,batch_size=128)
model.save("cifar10model.h5")
accuracy

10000/10000 [==============================] - 122s 12ms/step


[0.532594398021698, 0.8279]

# It keeps running more than 7h. It would be needed to implement some callback functions.

Las funciones de callback que vamos a usar son EarlyStopping, para que el entrenamiento pare si ve que no mejora la función de coste tras determinados epochs, y ReduceLROnPlateau, que si el entrenamiento no mejora tras unos epochs específicos, reduce el valor de learning rate del modelo, lo que normalmente obtiene una mejora del entrenamiento.

early_stop = EarlyStopping(monitor='val_loss', patience=8, verbose=1, min_delta=1e-4)  
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, verbose=1, min_delta=1e-4)  
callbacks_list = [early_stop, reduce_lr]  
model_history = model.fit_generator(  
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    callbacks=callbacks_list)

